In [ ]:
import numpy
import numpy as np
import pylab
import scipy
import cPickle
#from MyMultiSimTools import *
from os.path import expandvars
import copy
from icecube import icetray, dataclasses, dataio, clsim

from mpl_toolkits.axes_grid1 import make_axes_locatable


%matplotlib inline

In [ ]:
# Load the PPC ice model
PPCModel = numpy.loadtxt("./Dima_Icemodel.dat")
ScaPPC   = PPCModel[:-1,1]
AbsPPC   = PPCModel[:-1,2]


In [ ]:
def sample_coefficients(depths, a=0.15, b=0.15, correlation_length=15.):
        """                                                                                                                                                             
        Generate a proportional variation of scattering and absorption coefficients with                                                                                
        appropriate cross-correlations.                                                                                                                                 
                                                                                                                                                                        
        :param depths: array of layer depths                                                                                                                            
        :param a: standard deviation of absorption coefficients (w.r.t nominal values)                                                                                  
        :param b: standard deviation of scattering coefficients (w.r.t nominal values)                                                                                  
        :param correlation_length: scale of exponentially-decaying depth-dependent correlation in meters                                                                
        """
        # layer-to-layer correlations decrease exponentially                                                                                                            
        dd = abs(depths.reshape((depths.size, 1)).repeat(depths.size, axis=1) - depths.reshape((1, depths.size)).repeat(depths.size, axis=0))
        cov_a = a**2 * numpy.exp(-dd/correlation_length)
        cov_b = b**2 * numpy.exp(-dd/correlation_length)

        total_cov = numpy.zeros((2*depths.size, 2*depths.size))
        # a and b are individually correlated between layers                                                                                                            
        total_cov[::2,:][:,::2] = cov_a
        total_cov[1::2,:][:,1::2] = cov_b
        # a and b are totally (anti)correlated with each other within one layer                                                                                         
        crosscorr = -a*b
        for i in xrange(1,total_cov.shape[0]-1):
                total_cov[i,i+1] = total_cov[i,i-1] = crosscorr
        total_cov[0,1] = crosscorr
        total_cov[-1,-2] = crosscorr

        samples = numpy.random.multivariate_normal(mean=numpy.ones(2*depths.size), cov=total_cov, size=1)[0]
        sa = samples[::2]
        sb = samples[1::2]

        return sa, sb



In [ ]:
def sample_ice_properties(m, iceParams, anisotropyCoeffRelativeSigma=0.2, **kwargs):
        new_m = copy.deepcopy(m)
        newabs=[]
        newscat=[]

        if numpy.isnan(iceParams["anisotropyMagnitudeAlongDir"]) or numpy.isnan(iceParams["anisotropyMagnitudePerpToDir"]):
                raise RuntimeError("ice model does not have anisotropy!")

        depths = numpy.linspace(
                m.GetLayersZStart(),
                m.GetLayersZStart()+(m.GetLayersNum()-1)*m.GetLayersHeight(),
                m.GetLayersNum(),
                endpoint=True)


        sa, sb = sample_coefficients(depths, **kwargs)
    
        # replace the scattering and absorption length functions                                                                                                        
        for i in range(new_m.GetLayersNum()):
                oldScat = new_m.GetScatteringLength(i)
                oldAbs = new_m.GetAbsorptionLength(i)
                
                newScat = clsim.I3CLSimFunctionScatLenIceCube(
                        alpha = oldScat.alpha,
                        b400  = oldScat.b400 * sb[i]
                        )
                newabs.append(oldAbs.aDust400 * sa[i])
                newscat.append( oldScat.b400 * sb[i])

                newAbs = clsim.I3CLSimFunctionAbsLenIceCube(
                        kappa    = oldAbs.kappa,
                        A        = oldAbs.A,
                        B        = oldAbs.B,
                        D        = oldAbs.D,
                        E        = oldAbs.E,
                        aDust400 = oldAbs.aDust400 * sa[i],
                        deltaTau = oldAbs.deltaTau
                        )

                new_m.SetScatteringLength(i, newScat)
                new_m.SetAbsorptionLength(i, newAbs)

        # sample a pair of ice anisotropy parameters                                                                                                                    
        mean_coeffs = numpy.array([iceParams["anisotropyMagnitudeAlongDir"], iceParams["anisotropyMagnitudePerpToDir"]])
        sigmas = numpy.abs(mean_coeffs*anisotropyCoeffRelativeSigma)
        # print "original aniz coeffs are", mean_coeffs                                                                                                                 

        k1, k2 = numpy.random.multivariate_normal(mean=mean_coeffs, cov=numpy.array([[sigmas[0],0.], [0.,sigmas[1]]])**2, size=1)[0]
        # print "new aniz coeffs are", k1, k2                                                                                                                           

        newParams = copy.deepcopy(iceParams)
        newParams["anisotropyMagnitudeAlongDir"] = k1
        newParams["anisotropyMagnitudePerpToDir"] = k2

        newParams["absorptionScale"] = sa
        newParams["scatteringScale"] = sb

        absLenScaling, preScatterTransform, postScatterTransform = \
        clsim.util.GetSpiceLeaAnisotropyTransforms(
            newParams["anisotropyDirAzimuth"],
            newParams["anisotropyMagnitudeAlongDir"],
            newParams["anisotropyMagnitudePerpToDir"]
            )

        return (new_m, newParams,newabs,newscat)

In [ ]:
# Load the CLSim ice model
m, iceParams = clsim.MakeIceCubeMediumProperties(
                iceDataDirectory=expandvars("$I3_BUILD/clsim/resources/ice/spice_lea"),
                returnParameters=True)

depths = numpy.linspace(
                m.GetLayersZStart(),
                m.GetLayersZStart()+(m.GetLayersNum()-1)*m.GetLayersHeight(),
                m.GetLayersNum(),
                endpoint=True)

sa, sb = sample_coefficients(depths)


AnisotropyCLSim = numpy.array([iceParams["anisotropyMagnitudeAlongDir"], iceParams["anisotropyMagnitudePerpToDir"]])

DepthsCLSim = numpy.linspace(
        m.GetLayersZStart(),
        m.GetLayersZStart()+(m.GetLayersNum()-1)*m.GetLayersHeight(),
        m.GetLayersNum(),
        endpoint=True)
ScaCLSim=[]
AbsCLSim=[]
for i in range(len(DepthsCLSim)-1, 0,-1):
    ScaCLSim.append(m.GetScatteringLength(i).b400)
    AbsCLSim.append(m.GetAbsorptionLength(i).aDust400)

AbsScaScaleCLSim  = sum(AbsCLSim)/sum(ScaCLSim)
AbsCLSim          = numpy.array(AbsCLSim)
ScaCLSim          = numpy.array(ScaCLSim)
MinDepthCLSim     = numpy.array(DepthsCLSim[1])
DepthsCLSim       = DepthsCLSim[1:]-MinDepthCLSim


oldFFTAmp=np.abs(np.fft.rfft(np.log10(AbsCLSim*ScaCLSim)/2))
oldFFT=oldFFTAmp
oldFFTPhase=np.angle(np.fft.rfft(np.log10(AbsCLSim*ScaCLSim)/2))


In [ ]:
this_model, this_iceParams,newabs,newscat = sample_ice_properties(m, iceParams)



In [ ]:
pylab.plot(AbsPPC)
pylab.plot(ScaPPC)

In [ ]:
for i in range(0,10):
    pylab.figure(figsize=(7,4),dpi=200)
    this_model, this_iceParams,newabs,newscat = sample_ice_properties(m, iceParams)
    pylab.plot(AbsCLSim,'-',color='red',label='SpiceLea')
    pylab.plot(newabs[::-1],label='HESE Perturbaton ' +str(i))
    pylab.semilogy()
    pylab.ylim(1e-3,1e-1)
    pylab.legend(loc='lower left')
    pylab.ylabel("Absorption Coeff")
    pylab.xlabel("Layer")

#pylab.ylim(-0.1,0.5)

In [ ]:
for i in range(0,10):
    pylab.figure(figsize=(7,4),dpi=200)
    this_model, this_iceParams,newabs,newsca = sample_ice_properties(m, iceParams)
    pylab.plot(ScaCLSim,'-',color='red',label='SpiceLea')
    pylab.plot(newsca[::-1],label='HESE Perturbaton ' +str(i))
    pylab.semilogy()
   # pylab.ylim(1e-3,1e-1)
    pylab.legend(loc='lower left')
    pylab.ylabel("Scattering Coeff")
    pylab.xlabel("Layer")

In [ ]:
pylab.figure(figsize=(7,4),dpi=200)

pylab.plot(oldFFT,label='SpiceLea ')


for i in range(0,10):
    this_model, this_iceParams,newabs,newsca = sample_ice_properties(m, iceParams)
    newabs=np.array(newabs[::-1])
    newsca=np.array(newsca[::-1])
    newFFT=np.abs(np.fft.rfft(np.log10(newabs[:-1]*newsca[:-1])/2))
    pylab.plot(newFFT,'-',label='Pert. ' +str(i))
pylab.ylim(0.1,500)
pylab.xlim(0,76)
pylab.semilogy()
pylab.legend(loc='upper right',ncol=3)
pylab.ylabel("Amplitude")
pylab.xlabel("M+ Mode")



In [ ]:
pylab.figure(figsize=(7,4),dpi=200)

pylab.plot([0,20],[1,1],'--',color='red',label='SpiceLea')
#pylab.plot(oldFFT,label='SpiceLea ')
TotalAbs=[]
for i in range(0,10):
    this_model, this_iceParams,newabs,newsca = sample_ice_properties(m, iceParams)
    
    newabs=np.array(newabs[::-1])
    newsca=np.array(newsca[::-1])
    newFFT=np.abs(np.fft.rfft(np.log10(newabs[:-1]*newsca[:-1])/2))
    pylab.plot(newFFT/oldFFTAmp,'-',label='Pert. ' +str(i))
pylab.ylim(0.6,1.5)
pylab.xlim(0,10)
#pylab.semilogy()
pylab.legend(loc='upper right',ncol=3)
pylab.ylabel("Relative Amplitude Shift")
pylab.xlabel("M+ Mode")

In [ ]:
len(newsca)

In [ ]:
pylab.plot(newsca[:-1])
pylab.plot(newabs[:-1])
pylab.plot(AbsCLSim)
pylab.plot(ScaCLSim)

pylab.semilogy()
pylab.ylim(1e-3,10)
pylab.show()
pylab.plot(np.log10(newabs[:-1]*newsca[:-1])/2)
pylab.plot(np.log10(AbsCLSim*ScaCLSim)/2)

pylab.show()

pylab.plot(np.abs(np.fft.rfft(np.log10(newabs[:-1]*newsca[:-1])/2)))
pylab.plot(np.abs(np.fft.rfft(np.log10(AbsCLSim*ScaCLSim)/2)))
pylab.loglog()
pylab.show()

pylab.plot(np.abs(np.fft.rfft(np.log10(newabs[:-1]*newsca[:-1])/2))-np.abs(np.fft.rfft(np.log10(AbsCLSim*ScaCLSim)/2)))
pylab.ylim(-0.5,0.5)
pylab.xlim(0,12)

In [ ]:
pylab.figure(figsize=(7,4),dpi=200)

pylab.plot([0,20],[0,0],'--',color='red',label='SpiceLea')
#pylab.plot(oldFFT,label='SpiceLea ')


for i in range(0,10):
    this_model, this_iceParams,newabs,newsca = sample_ice_properties(m, iceParams)
    newabs=np.array(newabs[::-1])
    newsca=np.array(newsca[::-1])
    newFFTPhase=np.angle(np.fft.rfft(np.log10(newabs[:-1]*newsca[:-1])/2))
    pylab.plot(newFFTPhase-oldFFTPhase,'-',label='Pert. ' +str(i))
pylab.ylim(-0.3,0.3)
pylab.xlim(0,10)
#pylab.semilogy()
pylab.legend(loc='upper right',ncol=3)
pylab.ylabel("Phase Shift (rad)")
pylab.xlabel("M+ Mode")


In [ ]:
TotalAbs=[]

AmpShifts=[]
PhaseShifts=[]
Modes=10
Models=500
for i in range(0,Modes):
    AmpShifts.append([])
    PhaseShifts.append([])
for i in range(0,Models):
    this_model, this_iceParams,newabs,newsca = sample_ice_properties(m, iceParams)


    newabs=np.array(newabs[::-1])
    newsca=np.array(newsca[::-1])
    TotalAbs.append(np.average(newabs[:-1]/AbsCLSim))


    newFFT=np.abs(np.fft.rfft(np.log10(newabs[:-1]*newsca[:-1])/2))
    newFFTPhase=np.angle(np.fft.rfft(np.log10(newabs[:-1]*newsca[:-1])/2))
    for j in range(0,Modes):
        PhaseDiff=newFFTPhase[j]-oldFFTPhase[j]
        PhaseShifts[j].append(PhaseDiff)
        AmpShifts[j].append(newFFT[j]/oldFFT[j])


In [ ]:
pylab.figure(figsize=(5,5),dpi=200)
pylab.hist(TotalAbs/np.average(TotalAbs),bins=30)
pylab.xlabel("Average Absorption Shift")
pylab.ylabel("Count")
pylab.std(TotalAbs/np.average(TotalAbs))

In [ ]:
AmpSpreads=[]
AmpMeans=[]

PhaseSpreads=[]
PhaseMeans=[]
for i in range(0,Modes):
    AmpSpreads.append(np.std(AmpShifts[i]))
    AmpMeans.append(np.average(AmpShifts[i]))

    PhaseSpreads.append(np.std(PhaseShifts[i]))
    PhaseMeans.append(np.average(PhaseShifts[i]))

In [ ]:

MultisimAmpPriors=np.loadtxt("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/Constraints/Widths/AmpWidths.txt")
MultisimPhasePriors=np.loadtxt("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/Constraints/Widths/PhsWidths.txt")



In [ ]:
pylab.figure(figsize=(7,4),dpi=200)

pylab.plot([-0.5,10],[0,0],'--',color='black')
pylab.ylabel("Relative Amplitude uncertainty")
pylab.errorbar(np.arange(Modes)-0.1,np.array(AmpMeans)-1,yerr=AmpSpreads,capsize=5,fmt='o',color='DarkBlue',label='HESE toy models')
pylab.errorbar(MultisimAmpPriors[:,0]+0.1,MultisimAmpPriors[:,1],yerr=(MultisimAmpPriors[:,3]-MultisimAmpPriors[:,2])/2.,capsize=5,fmt='o',color='DarkRed',label='Multisim')
pylab.xlim(-0.5,10)



pylab.xlabel("M+ Mode")
pylab.legend(loc='upper left')
pylab.tight_layout()
pylab.savefig("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/Comparisons/Plots/HESECompAmp.png",dpi=300,bbox_inches='tight')

pylab.show()

pylab.figure(figsize=(7,4),dpi=200)
pylab.plot([-0.5,10],[0,0],'--',color='black')


pylab.ylabel("Phase uncertainty")
pylab.errorbar(np.arange(Modes)-0.1,np.array(PhaseMeans),yerr=PhaseSpreads,capsize=5,fmt='o',color='DarkBlue',label='HESE toy models')
pylab.errorbar(MultisimPhasePriors[:,0]+0.1,MultisimPhasePriors[:,1],yerr=(MultisimPhasePriors[:,3]-MultisimPhasePriors[:,2])/2.,capsize=5,fmt='o',color='DarkRed',label='Multisim')
pylab.legend(loc='upper left')
pylab.xlim(-0.5,10)
pylab.xlabel("M+ Mode")
pylab.legend(loc='upper left')
pylab.tight_layout()
pylab.savefig("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/Comparisons/Plots/HESECompPhs.png",dpi=300,bbox_inches='tight')


pylab.show()

In [ ]:
AmpWidth=(MultisimAmpPriors[:,3]-MultisimAmpPriors[:,2])[0:-1]
PhsWidth=(MultisimPhasePriors[:,3]-MultisimPhasePriors[:,2])[0:-1]

In [ ]:
Modes=5

In [ ]:
Models=1000
CovEvals=[]
ControlEvals=[]
NewCovMatrix=np.zeros_like(np.ndarray(shape=(2*Modes-1,2*Modes-1)))
FullPriors=numpy.concatenate([AmpWidth[0:Modes],PhsWidth[0:Modes-1]])

for i in range(0,Models):
    this_model, this_iceParams,newabs,newsca = sample_ice_properties(m, iceParams)
    newabs=np.array(newabs[::-1])
    newsca=np.array(newsca[::-1])
    newFFT=np.abs(np.fft.rfft(np.log10(newabs[:-1]*newsca[:-1])/2))
    newFFTPhase=np.angle(np.fft.rfft(np.log10(newabs[:-1]*newsca[:-1])/2))
    
    NuissanceVector1=(newFFT/oldFFT-1.)[0:5]
    NuissanceVector2=(newFFTPhase-oldFFTPhase)[1:5]
    NuissanceVectorFull=numpy.concatenate([NuissanceVector1,NuissanceVector2])/FullPriors
   # CovEval=np.dot(np.dot(np.transpose(NuissanceVectorFull),FullHessian),NuissanceVectorFull)
   # CovEvals.append(CovEval)
    
  #  ARealization=numpy.random.multivariate_normal(numpy.zeros(21),CovNuis)

    for a in range(0,2*Modes-1):
        for b in range(0,2*Modes-1):
            NewCovMatrix[a,b]+=NuissanceVectorFull[a]*NuissanceVectorFull[b]


In [ ]:
NewCovMatrix/=Models
ControlEvals=np.array(ControlEvals)
CovEvals=np.array(CovEvals)

In [ ]:
from scipy import stats

In [ ]:
np.savetxt("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/Comparisons/CovMatrixHESEToys.txt",NewCovMatrix)


In [ ]:
pylab.figure(figsize=(4,4),dpi=300)   


im =pylab.imshow(NewCovMatrix,cmap='RdBu')
pylab.clim(-1,1)
pylab.plot([-0.5,2.*Modes-1.5],[Modes-0.5,Modes-0.5],'--',color='black')
pylab.plot([Modes-0.5,Modes-0.5],[-0.5,2.*Modes-1.5],'--',color='black')

XTickNames=[]
for i in range(0,Modes):
    XTickNames.append("Amp "+str(i))
for i in range(1,Modes):
    XTickNames.append("Phs "+str(i))
    
pylab.xticks(range(2*Modes+1),XTickNames,rotation=90)
pylab.yticks(range(2*Modes+1),XTickNames)
pylab.xlim(-0.5,2*Modes-1.5)
pylab.ylim(2*Modes-1.5,-0.5)
ax=pylab.gca()

ax.xaxis.tick_top()
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)

pylab.colorbar(im, cax=cax)
pylab.tight_layout()

pylab.savefig("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/Comparisons/Plots/NuisanceCovariance",dpi=250,bbox_inches='tight')

